In [1]:
import pandas as pd
import keras
from sklearn import preprocessing

Using TensorFlow backend.


In [2]:
data_supervised = pd.read_pickle('data_supervised.pkl.gzip').drop(['index'],axis = 1)

In [3]:
data_supervised['ingreso_max'] = data_supervised['ingreso_max'].apply(lambda x: min(x,100000000))

In [4]:
data_supervised.head()

,id_trn_ach,id_cliente,valor_trx,fecha_usable,trx_mes_corrido,seg_str,ocupacion,tipo_vivienda,nivel_academico,estado_civil,...,categoria_4_clusters,nombre,key1,key2,key3,cat_target,cat_past1,cat_past2,cat_past3,final_nlp
0,215120522,11856,32229.02,2016-09-01 00:00:59,32229.02,PERSONAL,1,O,T,S,...,2,platino,adulto,acomodado,tecnico,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo,vacio,vacio"
1,215120586,241516,26804.07,2016-09-01 00:02:44,26804.07,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo,vacio,vacio"
2,215120682,206803,270247.63,2016-09-01 00:04:45,270247.63,PERSONAL,1,F,E,M,...,2,platino,adulto,acomodado,tecnico,Hogar,NaN,NaN,NaN,"empresas,publicas,medellin,esp,cc,cc"
3,215120662,44531,78317.39,2016-09-01 00:04:50,78317.39,PERSONAL,1,I,U,S,...,0,universitario,joven,pobre,estudiante,Tecnología y comunicaciones,NaN,NaN,NaN,"pago,saldo,vacio,vacio"
4,215120740,135203,1063830.22,2016-09-01 00:07:41,1642590.43,PERSONAL PLUS,1,F,U,M,...,1,black,mayor,rico,profesional,Servicios financieros,NaN,NaN,NaN,"pago,factura,asociado,bancoomevapfa,bancoomevapfa"


In [5]:
data_supervised.columns

Index(['id_trn_ach', 'id_cliente', 'valor_trx', 'fecha_usable',
       'trx_mes_corrido', 'seg_str', 'ocupacion', 'tipo_vivienda',
       'nivel_academico', 'estado_civil', 'genero', 'ingreso_rango',
       'ingreso_min', 'ingreso_max', 'edad_v2', 'categoria_4_clusters',
       'nombre', 'key1', 'key2', 'key3', 'cat_target', 'cat_past1',
       'cat_past2', 'cat_past3', 'final_nlp'],
      dtype='object')

# Se eliminan transacciones extranas

In [6]:
data_supervised.valor_trx.value_counts().sort_index()

1.000000e-02     2
4.000000e-02     1
9.000000e-02     1
1.000000e-01     1
1.500000e-01     1
1.700000e-01     2
1.900000e-01     1
3.600000e-01     1
4.500000e-01     1
6.100000e-01     1
6.900000e-01     1
8.300000e-01     1
9.300000e-01     1
9.500000e-01    12
9.600000e-01    21
9.700000e-01    25
9.800000e-01    26
9.900000e-01    29
1.000000e+00    22
1.010000e+00    21
1.020000e+00    22
1.030000e+00    26
1.040000e+00    23
1.050000e+00    11
1.190000e+00     1
1.560000e+00     1
1.650000e+00     1
1.900000e+00     1
1.920000e+00     2
1.930000e+00     5
                ..
6.987331e+07     1
7.140118e+07     1
7.160155e+07     1
7.235086e+07     1
7.262303e+07     1
7.489004e+07     1
7.669307e+07     1
8.145455e+07     1
8.293639e+07     1
8.870910e+07     1
9.414391e+07     1
9.447256e+07     1
9.467762e+07     1
9.762498e+07     1
9.964566e+07     1
1.025643e+08     1
1.058366e+08     1
1.059129e+08     1
1.075865e+08     1
1.077696e+08     1
1.299421e+08     1
1.394687e+08

# Todo lo que este por encima de 5 millones se elimina. A su vez a todo lo que este por debajo de 100 peso.

In [9]:
arriba_peso = data_supervised.valor_trx >100
abajo_50m = data_supervised.valor_trx <50000001
sin_ingresos = pd.isnull(data_supervised.ingreso_min)

In [10]:
data_supervised = data_supervised[(abajo_50m) & (arriba_peso) & (~sin_ingresos)]

In [11]:
data_supervised.shape

(3195528, 25)

In [12]:
data_supervised = data_supervised.reset_index().drop('index',axis=1)

In [13]:
dummy_data_cat = pd.get_dummies(data_supervised[['seg_str', 'ocupacion', 'tipo_vivienda',
       'nivel_academico', 'estado_civil', 'genero', 'ingreso_rango','categoria_4_clusters',
       'nombre', 'key1', 'key2', 'key3','cat_past1',
       'cat_past2', 'cat_past3']])

In [14]:
dummy_data_cat.shape

(3195528, 148)

# Prepara el texto

In [15]:
from numpy import array
from keras.preprocessing.text import one_hot
docs = data_supervised['final_nlp']
# integer encode the documents
vocab_size = 15000
encoded_docs = [one_hot(d, vocab_size) for d in docs]

In [16]:
# Truncate and pad the review sequences 
from keras.preprocessing import sequence 
max_texto_pse = 20 
nlp_features = sequence.pad_sequences(encoded_docs, maxlen=max_texto_pse) 

In [17]:
nlp_features.shape

(3195528, 20)

# Continuous features

In [18]:
data_supervised_cont = data_supervised[['valor_trx',
       'trx_mes_corrido','ingreso_min', 'ingreso_max']]

In [19]:
robust_train = preprocessing.RobustScaler()
minmax = preprocessing.MinMaxScaler()

Transformo la variable de aprendizaje

In [20]:
num_classes = 27
le = preprocessing.LabelEncoder()
le.fit(data_supervised['cat_target'].unique())
y_labeled = le.transform(data_supervised['cat_target'])
y = keras.utils.to_categorical(y_labeled, num_classes)

# Split de aprendizaje

In [21]:
from sklearn.model_selection import train_test_split
index_info_train, index_info_test, Y_train, Y_test = train_test_split(data_supervised_cont, y, test_size = 0.2, random_state=42)

In [22]:
index_train = index_info_train.index
index_test = index_info_test.index

# Creacion de tablas

In [23]:
dummy_train = dummy_data_cat.loc[index_train]
dummy_test = dummy_data_cat.loc[index_test]
data_supervised_cont_train = data_supervised_cont.loc[index_train]
data_supervised_cont_test = data_supervised_cont.loc[index_test]
nlp_features_train = nlp_features[index_train,:]
nlp_features_test = nlp_features[index_test,:]

# Preprocesado de data continua

In [25]:
data_supervised_cont_train_ready = minmax.fit_transform(robust_train.fit_transform(data_supervised_cont_train))
data_supervised_cont_test_ready = minmax.transform(robust_train.transform(data_supervised_cont_test))

In [29]:
data_supervised_cont_train_ready.shape

(2556422, 4)

In [30]:
nlp_features_train.shape

(2556422, 20)

In [31]:
dummy_train.shape

(2556422, 148)

# Construccion de la red

In [40]:
from keras.layers import Dense,Dropout, LSTM, Input, Embedding,Flatten, concatenate
from keras.models import Model

In [52]:
contin_input = Input(shape=(4, ))
nlp_input = Input(shape=(20,) )
one_hot_input = Input(shape=(148,) )

embedding_nlp = Embedding(vocab_size, 32, input_length=max_texto_pse)(nlp_input)
#lstm_embedding = LSTM(16)(embedding_nlp)
dense_embedding = Dense(32)(embedding_nlp)


cont_dense = Dense(32)(contin_input)
one_hot_dense = Dense(32)(one_hot_input)

drop_out_embedding = Dropout(0.20)(dense_embedding)
drop_out_cont = Dropout(0.20)(cont_dense)
drop_out_one_hot = Dropout(0.20)(one_hot_dense)


merge_one = concatenate([drop_out_cont, drop_out_one_hot])

#merge_two = concatenate([merge_one, drop_out_embedding])

Dense_relu = Dense(32, activation='relu')(merge_one)
#Embedding_last = Dense(32, activation='relu')(dense_embedding)

#merge_two = concatenate([Dense_relu, Embedding_last])

final_drop = Dropout(0.25)(Dense_relu)
dense_output = Dense(num_classes,activation='sigmoid')(final_drop)



model = Model(inputs=[contin_input, nlp_input,one_hot_input], outputs=dense_output)


model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer='adam',
              metrics=['accuracy'])

model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_46 (InputLayer)           (None, 4)            0                                            
__________________________________________________________________________________________________
input_48 (InputLayer)           (None, 148)          0                                            
__________________________________________________________________________________________________
dense_57 (Dense)                (None, 32)           160         input_46[0][0]                   
__________________________________________________________________________________________________
dense_58 (Dense)                (None, 32)           4768        input_48[0][0]                   
__________________________________________________________________________________________________
dropout_48

In [ ]:
callback_keras = keras.callbacks.TensorBoard(log_dir='./Graph', histogram_freq=0,  
          write_graph=True, write_images=True)

model.fit([data_supervised_cont_train_ready,nlp_features_train,dummy_train], Y_train, epochs=2,callbacks=[callback_keras])

# Final evaluation of the model
scores = model.evaluate([data_supervised_cont_test_ready,nlp_features_test,dummy_test], Y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/2
 378176/2556422 [===>..........................] - ETA: 4:26 - loss: 0.8258 - acc: 0.7469

# Lamentablemente por un tema de tiempos la arquitectura no es la optima, y deseariamos trabajarle ma. 